In [1]:
import numpy as np

def generate_pwl_with_spikes(spike_times, start_time, end_time, time_step=0.05, filename='output.pwl'):
    # Spike pattern (based on your provided data)
    spike_pattern = [
    (-0.0418396, 0.0),
    (-0.0416107, 0.05),
    (-0.0413574, 0.1),
    (-0.0410522, 0.15),
    (-0.0402985, 0.2),
    (-0.0372772, 0.25),
    (-0.030658, 0.3),
    (-0.0194275, 0.35),
    (-0.00132141, 0.4),
    (0.0177612, 0.45),
    (0.0303101, 0.5),
    (0.0360443, 0.55),
    (0.0373871, 0.6),
    (0.0360107, 0.65),
    (0.032959, 0.7),
    (0.0288055, 0.75),
    (0.0240082, 0.8),
    (0.0190521, 0.85),
    (0.0142456, 0.9),
    (0.00965271, 0.95),
    (0.0053772, 1.0),
    (0.00144958, 1.05),
    (-0.00219421, 1.1),
    (-0.00560303, 1.15),
    (-0.00877991, 1.2),
    (-0.0117462, 1.25),
    (-0.0145264, 1.3),
    (-0.0171021, 1.35),
    (-0.0194672, 1.4),
    (-0.0215881, 1.45),
    (-0.0234894, 1.5),
    (-0.0251953, 1.55),
    (-0.0266937, 1.6),
    (-0.028009, 1.65),
    (-0.0291901, 1.7),
    (-0.0302277, 1.75)
]

    # Total time range
    total_time = np.arange(start_time, end_time + time_step, time_step)
    spike_duration = spike_pattern[-1][1]  # Duration of the spike pattern in ms

    # Function to determine if we're in a spike range
    def is_spike_time(current_time):
        for spike_time in spike_times:
            if spike_time <= current_time < spike_time + spike_duration:
                return spike_time
        return None

    # Open file for writing the PWL
    with open(filename, 'w') as pwl_file:
        for t in total_time:
            spike_start = is_spike_time(t)
            if spike_start is not None:
                # Determine where we are in the spike pattern
                time_in_spike = t - spike_start
                # Find the voltage at this time in the spike pattern
                spike_voltage = next(v for v, dt in spike_pattern if dt == round(time_in_spike, 3))
                pwl_file.write(f"{t}ms\t{spike_voltage}V\n")
            else:
                # If not in a spike period, use the default voltage
                pwl_file.write(f"{t}ms\t-0.038V\n")



def generate_square_pulses(timestamps, pulse_width, start_time, end_time, 
                           default_voltage=0, V_on=1.2, time_step=0.05, filename='square_pulses.pwl'):
    # Create time array
    total_time = np.arange(start_time, end_time + time_step, time_step)
    
    # Function to determine the voltage at a given time
    def get_voltage_at_time(t):
        for pulse_start in timestamps:
            if pulse_start <= t < pulse_start + pulse_width:
                return V_on
        return default_voltage
    
    # Write the PWL file
    with open(filename, 'w') as pwl_file:
        for t in total_time:
            voltage = get_voltage_at_time(t)
            pwl_file.write(f"{t}ms\t{voltage}V\n")
    
    print(f"PWL file '{filename}' generated successfully.")




In [20]:
# Example usage:
spike_times = [1, 3, 3, 20, 25]  # Example spike times
generate_pwl_with_spikes(spike_times, start_time=0, end_time=30, time_step=0.05, filename='spike_output.pwl')

# Example usage:
timestamps = [5, 12, 20]  # Pulse start times in ms
pulse_width = 0.1  # Pulse width in ms
generate_square_pulses(timestamps, pulse_width, start_time=0, end_time=25, time_step=0.05)


PWL file 'square_pulses.pwl' generated successfully.


In [23]:


spike_series = [
    [1,3],
    [1,4],
    [4.5],
    [1],
    [2],
    [3],
    [4],
]

# Flatten the list of lists and add some constant with max value + 1 to each element

final_spike_series = []

current_sublist_end_time = 0
for sublist in spike_series:
    for i in sublist:
        final_spike_series.append(i + current_sublist_end_time)
    current_sublist_end_time += 5  #max(sublist)



print(f"Final spike series: {final_spike_series}")

# Now generate control measure pulses at the end of each 5ms interval

pulse_width = 0.1  # Pulse width in ms

pulse_timestamps = [5*i+5 for i in range(len(spike_series))]

discharge_timestamps = [5*i+0.2 for i in range(len(spike_series))]

print(f"Pulse timestamps: {pulse_timestamps}")

# Generate pulses and spikes
generate_square_pulses(pulse_timestamps, pulse_width, start_time=0, end_time=40, time_step=0.05, filename='control_pulses.pwl.txt')
generate_pwl_with_spikes(final_spike_series, start_time=0, end_time=40, time_step=0.05, filename='spike_output.pwl.txt')
generate_square_pulses(discharge_timestamps, pulse_width, start_time=0, end_time=40, time_step=0.05, filename='discharge_pulses.pwl.txt')


Final spike series: [1, 3, 6, 9, 14.5, 16, 22, 28, 34]
Pulse timestamps: [5, 10, 15, 20, 25, 30, 35]
PWL file 'control_pulses.pwl.txt' generated successfully.
PWL file 'discharge_pulses.pwl.txt' generated successfully.


PWL file 'square_pulses.pwl' generated successfully.
